# 📥 Notebook 1: Coleta de Dados - X/Twitter e Reddit

Este notebook demonstra o processo de coleta de dados de redes sociais (X/Twitter e Reddit) para análise de sentimento em tópicos específicos.

## 🎯 Objetivos
- Configurar e testar coletores de dados
- Coletar posts sobre um tópico específico
- Validar qualidade dos dados coletados
- Preparar dados para pré-processamento

## 📋 Pré-requisitos
- Credenciais do Reddit configuradas em `.env`
- Ambiente Python com dependências instaladas
- Configuração do tópico em `config/topic.yaml`

In [ ]:
# Imports e configuração inicial
import sys
import os
import pandas as pd
import yaml
from datetime import datetime
from pathlib import Path
import logging

# Adicionar src ao path
sys.path.append('../src')

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Verificar estrutura do projeto
project_root = Path("..").resolve()
print(f"📁 Diretório do projeto: {project_root}")
print(f"📂 Estrutura principal:")
for item in ["config", "src", "data", "scripts"]:
    path = project_root / item
    status = "✅" if path.exists() else "❌"
    print(f"  {status} {item}/")

print("\n🔧 Python path:")
for p in sys.path[-3:]:
    print(f"  - {p}")

In [ ]:
# Carregar configuração do tópico
config_path = project_root / "config" / "topic.yaml"

try:
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    
    print("📋 Configuração carregada:")
    print(f"  🎯 Tópico: {config['topic']}")
    print(f"  🔍 Keywords: {config['keywords']}")
    print(f"  📊 Limites:")
    print(f"    - Twitter: {config['limits']['twitter']}")
    print(f"    - Reddit: {config['limits']['reddit']}")
    print(f"  🔧 Filtros:")
    print(f"    - Comprimento: {config['filters']['min_length']}-{config['filters']['max_length']}")
    print(f"    - Idioma: {config['filters']['language']}")
    
except FileNotFoundError:
    print(f"❌ Arquivo de configuração não encontrado: {config_path}")
    print("💡 Execute: cp config/topic.yaml.example config/topic.yaml")
except Exception as e:
    print(f"❌ Erro ao carregar configuração: {e}")

In [ ]:
# Testar importação dos scrapers
try:
    from scrapers.twitter import TwitterScraper
    print("✅ TwitterScraper importado com sucesso")
except ImportError as e:
    print(f"❌ Erro ao importar TwitterScraper: {e}")

try:
    from scrapers.reddit import RedditScraper  
    print("✅ RedditScraper importado com sucesso")
except ImportError as e:
    print(f"❌ Erro ao importar RedditScraper: {e}")

try:
    from utils.config import load_config, setup_logging
    print("✅ Utilitários importados com sucesso")
except ImportError as e:
    print(f"❌ Erro ao importar utilitários: {e}")

# Verificar se todos os arquivos necessários existem
required_files = [
    "src/scrapers/twitter.py",
    "src/scrapers/reddit.py", 
    "src/utils/config.py",
    ".env.example"
]

print("\n📄 Verificando arquivos necessários:")
for file_path in required_files:
    full_path = project_root / file_path
    status = "✅" if full_path.exists() else "❌"
    print(f"  {status} {file_path}")

## 🐦 Teste de Coleta: X/Twitter

Vamos testar a coleta de dados do X/Twitter usando `snscrape` com um limite pequeno para validação.

In [ ]:
# Teste de coleta X/Twitter com limite pequeno
try:
    from scrapers.twitter import TwitterScraper
    
    # Configurações de teste
    test_keywords = config['keywords'][:2]  # Usar apenas 2 keywords para teste
    test_limit = 10  # Limite pequeno para teste
    
    print(f"🔍 Testando coleta com keywords: {test_keywords}")
    print(f"📊 Limite: {test_limit} tweets")
    
    # Inicializar scraper
    twitter_scraper = TwitterScraper()
    
    # Realizar coleta de teste
    tweets_data = twitter_scraper.collect(
        keywords=test_keywords,
        limit=test_limit,
        lang='pt'
    )
    
    if tweets_data:
        print(f"✅ Coleta bem-sucedida! {len(tweets_data)} tweets coletados")
        
        # Converter para DataFrame para análise
        df_tweets = pd.DataFrame(tweets_data)
        print(f"📊 Colunas: {list(df_tweets.columns)}")
        print(f"📝 Exemplo de tweet:")
        if len(df_tweets) > 0:
            print(f"  Texto: {df_tweets.iloc[0]['text'][:100]}...")
            print(f"  Data: {df_tweets.iloc[0]['timestamp']}")
            print(f"  Usuário: {df_tweets.iloc[0].get('user_hash', 'N/A')}")
    else:
        print("⚠️  Nenhum tweet coletado. Verifique as keywords ou conexão.")
        
except Exception as e:
    print(f"❌ Erro na coleta do Twitter: {e}")
    print("💡 Verifique se o snscrape está instalado: pip install snscrape")

## 🔴 Teste de Coleta: Reddit

Vamos testar a coleta de dados do Reddit usando PRAW. **Importante**: Este teste requer credenciais configuradas no arquivo `.env`.

In [ ]:
# Verificar credenciais do Reddit
import os
from dotenv import load_dotenv

# Carregar variáveis do .env
env_path = project_root / ".env"
if env_path.exists():
    load_dotenv(env_path)
    print("✅ Arquivo .env encontrado")
else:
    print("⚠️  Arquivo .env não encontrado")
    print("💡 Execute: cp .env.example .env e configure as credenciais")

# Verificar se as credenciais estão configuradas
required_vars = ["REDDIT_ID", "REDDIT_SECRET", "REDDIT_AGENT", "REDDIT_USERNAME", "REDDIT_PASSWORD"]
missing_vars = []

print("\n🔐 Verificando credenciais do Reddit:")
for var in required_vars:
    value = os.getenv(var)
    if value:
        # Mostrar apenas primeiros caracteres para segurança
        display_value = value[:4] + "..." if len(value) > 4 else value
        print(f"  ✅ {var}: {display_value}")
    else:
        print(f"  ❌ {var}: não configurado")
        missing_vars.append(var)

if missing_vars:
    print(f"\n⚠️  Variáveis faltando: {missing_vars}")
    print("💡 Configure no arquivo .env antes de continuar")
else:
    print("\n✅ Todas as credenciais estão configuradas!")

In [ ]:
# Teste de coleta Reddit com limite pequeno
try:
    from scrapers.reddit import RedditScraper
    
    # Verificar se as credenciais estão disponíveis
    if not missing_vars:
        print("🔍 Testando coleta do Reddit...")
        
        # Configurações de teste
        test_subreddits = config['reddit']['subreddits'][:1]  # Usar apenas 1 subreddit para teste
        test_limit = 5  # Limite pequeno para teste
        
        print(f"📊 Subreddits: {test_subreddits}")
        print(f"📊 Limite: {test_limit} posts por subreddit")
        
        # Inicializar scraper
        reddit_scraper = RedditScraper()
        
        # Realizar coleta de teste
        reddit_data = reddit_scraper.collect(
            keywords=config['keywords'][:2],
            subreddits=test_subreddits,
            limit=test_limit
        )
        
        if reddit_data:
            print(f"✅ Coleta bem-sucedida! {len(reddit_data)} posts coletados")
            
            # Converter para DataFrame para análise
            df_reddit = pd.DataFrame(reddit_data)
            print(f"📊 Colunas: {list(df_reddit.columns)}")
            print(f"📝 Exemplo de post:")
            if len(df_reddit) > 0:
                print(f"  Título: {df_reddit.iloc[0]['title'][:100]}...")
                print(f"  Data: {df_reddit.iloc[0]['timestamp']}")
                print(f"  Subreddit: {df_reddit.iloc[0].get('subreddit', 'N/A')}")
                print(f"  Score: {df_reddit.iloc[0].get('score', 'N/A')}")
        else:
            print("⚠️  Nenhum post coletado. Verifique as keywords ou subreddits.")
            
    else:
        print("❌ Não é possível testar Reddit sem credenciais configuradas")
        print("💡 Configure o arquivo .env primeiro")
        
except Exception as e:
    print(f"❌ Erro na coleta do Reddit: {e}")
    print("💡 Verifique se o praw está instalado: pip install praw")

## 🚀 Coleta Completa

Uma vez que os testes funcionaram, podemos executar a coleta completa usando os limites definidos na configuração.

In [ ]:
# Executar coleta completa usando o script collector.py
import subprocess
import json
from datetime import datetime

# Definir parâmetros da coleta
twitter_limit = config['limits']['twitter']
reddit_limit = config['limits']['reddit']
topic = config['topic']

print(f"🎯 Iniciando coleta completa para: {topic}")
print(f"📊 Limites: Twitter={twitter_limit}, Reddit={reddit_limit}")

# Criar timestamp para esta execução
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = project_root / "data" / "raw" / timestamp

print(f"📁 Salvando em: {output_dir}")

# Comando para executar o collector
cmd = [
    "python", 
    str(project_root / "scripts" / "collector.py"),
    "--config", str(project_root / "config" / "topic.yaml"),
    "--limit-twitter", str(twitter_limit),
    "--limit-reddit", str(reddit_limit),
    "--output-dir", str(output_dir),
    "--verbose"
]

print(f"🔧 Comando: {' '.join(cmd)}")
print("\n" + "="*50)
print("🚀 EXECUTANDO COLETA...")
print("="*50)

try:
    # Executar o comando
    result = subprocess.run(cmd, capture_output=True, text=True, cwd=str(project_root))
    
    print("📤 SAÍDA:")
    print(result.stdout)
    
    if result.stderr:
        print("⚠️  ERROS/AVISOS:")
        print(result.stderr)
    
    if result.returncode == 0:
        print("✅ Coleta concluída com sucesso!")
        
        # Verificar arquivos gerados
        if output_dir.exists():
            files = list(output_dir.glob("*.csv"))
            print(f"📄 Arquivos gerados: {len(files)}")
            for file in files:
                size_mb = file.stat().st_size / (1024*1024)
                print(f"  - {file.name}: {size_mb:.2f} MB")
        else:
            print("⚠️  Diretório de saída não encontrado")
    else:
        print(f"❌ Coleta falhou com código: {result.returncode}")
        
except Exception as e:
    print(f"❌ Erro ao executar coleta: {e}")

print("\n" + "="*50)